In [1]:
%pip install openai pydantic python-dotenv openai instructor graphviz eval_type_backport --quiet

import os
import openai
import instructor
from pydantic import BaseModel
from typing import List
from dotenv import load_dotenv
import graphviz
from IPython.display import Image, display

load_dotenv()
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")

Note: you may need to restart the kernel to use updated packages.


/Users/rohanrichard/Documents/code/knowledge-graph-agent/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
class Node(BaseModel):
    label: str
    entity_type: str


class Edge(BaseModel):
    source: Node
    target: Node
    label: str


class Graph(BaseModel):
    nodes: List[Node]
    edges: List[Edge]


class InferenceRequest(BaseModel):
    text: str
    graph: Graph


class InferenceResponse(BaseModel):
    response: str
    new_nodes: List[Node]
    new_edges: List[Edge]